In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

# Load the dataset
train_data = pd.read_csv('final_train.csv')
test_data = pd.read_csv('final_test.csv')

# Tokenize text data using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_len = 100  # Maximum sequence length
X_train_text = tokenizer(train_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')
X_test_text = tokenizer(test_data['text'].tolist(), padding='max_length', truncation=True, max_length=max_len, return_tensors='tf')

# Convert labels to numerical format
label_mapping = {'bully-Spam': 0, 'not_bully-Spam': 1, 'bully-Ham': 2, 'not_bully-Ham': 3}
y_train = train_data['new_label'].map(label_mapping)
y_test = test_data['new_label'].map(label_mapping)

# Define DistilBERT model
def build_distilbert_model():
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    distilbert_model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
    distilbert_outputs = distilbert_model(input_ids, attention_mask=attention_mask)[0]

    distilbert_pooled_output = GlobalAveragePooling1D()(distilbert_outputs)

    return Model(inputs=[input_ids, attention_mask], outputs=distilbert_pooled_output)

# Build DistilBERT model
distilbert_model = build_distilbert_model()

# Combine DistilBERT and DNN models
def build_combined_model(distilbert_model):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    distilbert_output = distilbert_model([input_ids, attention_mask])
    
    output = Dense(4, activation='softmax')(distilbert_output)

    return Model(inputs=[input_ids, attention_mask], outputs=output)

# Build combined model
combined_model = build_combined_model(distilbert_model)

# Compile combined model with a smaller learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Adjust the learning rate as needed
combined_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train combined model with a smaller batch size
history = combined_model.fit([X_train_text['input_ids'], X_train_text['attention_mask']], y_train, epochs=5, batch_size=16, validation_split=0.1)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Epoch 1/5
79/79 [==============================] - 174s 2s/step - loss: 0.7094 - accuracy: 0.7232 - val_loss: 0.5561 - val_accuracy: 0.7801
Epoch 2/5
79/79 [==============================] - 169s 2s/step - loss: 0.3506 - accuracy: 0.8787 - val_loss: 0.4850 - val_accuracy: 0.8298
Epoch 3/5
79/79 [==============================] - 170s 2s/step - loss: 0.1951 - accuracy: 0.9405 - val_loss: 0.1641 - val_accuracy: 0.9645
Epoch 4/5
79/79 [==============================] - 170s 2s/step - loss: 0.1099 - accuracy: 0.9643 - val_loss: 0.1799 - val_accuracy: 0.9291
Epoch 5/5
79/79 [==============================] - 170s 2s/step - loss: 0.0587 - accuracy: 0.9849 - val_loss: 0.2192 - val_accuracy: 0.9220


In [2]:
# Make predictions on test data
predictions = combined_model.predict([X_test_text['input_ids'], X_test_text['attention_mask']])

# Convert predicted probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Evaluate the model
from sklearn.metrics import classification_report

# Convert test labels to numpy array
y_test_np = np.array(y_test)

# Print classification report
print(classification_report(y_test_np, predicted_labels))

14/14 [==============================] - 17s 1s/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.93      0.92      0.93       115
           2       0.83      0.87      0.85       103
           3       0.89      0.87      0.88       198

    accuracy                           0.89       426
   macro avg       0.91      0.92      0.91       426
weighted avg       0.89      0.89      0.89       426

